In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import h5py
import copy

import bilby

# Preamble

In [2]:
# The population files -- use the uniform masses for now
BBH_pop_filename = "./18321_1yrCatalogBBH.h5"
BNS_pop_filename = "./18321_1yrCatalogBNS.h5"
BNS_gauss_pop_filename = "./18321_1yrCatalogBNSmassGauss.h5"
FILENAMES_DICT = {"BBH": BBH_pop_filename, 
                  "BNS": BNS_pop_filename, 
                  "BNS_gauss": BNS_gauss_pop_filename}

# Parameters that are in these files
ALL_BBH_KEYS = ['Mc', 'Phicoal', 'chi1', 'chi1x', 'chi1y', 'chi1z', 'chi2', 'chi2x', 'chi2y', 'chi2z', 'dL', 'dec', 'eta', 'iota', 'm1_source', 'm2_source', 'phi', 'phi12', 'phiJL', 'psi', 'ra', 'tGPS', 'tcoal', 'theta', 'thetaJN', 'tilt1', 'tilt2', 'z']
ALL_BNS_KEYS = copy.deepcopy(ALL_BBH_KEYS)
ALL_BNS_KEYS += ["Lambda1", "Lambda2"]

ALL_KEYS_DICT = {"BBH": ALL_BBH_KEYS, 
                 "BNS": ALL_BNS_KEYS, 
                 "BNS_gauss": ALL_BNS_KEYS}

In [3]:
def translate_for_bilby(params: dict):
    """Simply add the required key names for bilby"""
    
    params["luminosity_distance"] = params["dL"] * 1000.0 # from Gpc to Mpc
    params["mass_1"] = params["m1_source"]
    params["mass_2"] = params["m2_source"]
    
    params["tilt_1"] = params["tilt1"]
    params["tilt_2"] = params["tilt2"]
    
    params["theta_jn"] = params["thetaJN"]
    
    params["a_1"] = params["chi1"]
    params["a_2"] = params["chi2"]
    
    params["phase"] = params["phi"]
    
    params["geocent_time"] = params["tGPS"]
    
    return params

# Load and organize the data

In [4]:
CoBa_events_dict = {}
CoBa_events_dict["BBH"] = {}
CoBa_events_dict["BNS"] = {}

# Initialize the dicts with the keys
for pop_str in ["BBH", "BNS"]:
    for key in ALL_KEYS_DICT[pop_str]:
        CoBa_events_dict[pop_str][key] = []

for pop_str in ["BBH", "BNS"]:
    filename = FILENAMES_DICT[pop_str]
    with h5py.File(filename, "r") as file:
        for key in file.keys():
            CoBa_events_dict[pop_str][key] = np.array(file[key])

In [5]:
# CoBa_events_dict["BBH"]

# Compute SNRs

Pick a signal

In [6]:
idx = 1
example_signal = {}
for key in CoBa_events_dict["BBH"].keys():
    example_signal[key] = CoBa_events_dict["BBH"][key][idx]
    
example_signal = translate_for_bilby(example_signal)
example_signal

{'Mc': 20.456367915725753,
 'Phicoal': 0.4002946072018097,
 'chi1': 0.1802,
 'chi1x': -0.17407092992335507,
 'chi1y': -0.019180881891998174,
 'chi1z': -0.042466988655467684,
 'chi2': 0.1681,
 'chi2x': -0.013494913744330804,
 'chi2y': 0.1628069865637605,
 'chi2z': -0.039615431703574455,
 'dL': 1.8525396,
 'dec': -1.107836276297569,
 'eta': 0.24933982092425372,
 'iota': 1.2613238970364047,
 'm1_source': 18.465,
 'm2_source': 16.66,
 'phi': 2.733968915528476,
 'phi12': 4.6853416741266205,
 'phiJL': 5.741156800564721,
 'psi': 0.5917657886321974,
 'ra': 2.733968915528476,
 'tGPS': 1886480670,
 'tcoal': 0.3379993656387555,
 'theta': 2.6786326030924656,
 'thetaJN': 1.2473186485378833,
 'tilt1': 1.8087,
 'tilt2': 1.8087,
 'z': 0.3401,
 'luminosity_distance': 1852.5396,
 'mass_1': 18.465,
 'mass_2': 16.66,
 'tilt_1': 1.8087,
 'tilt_2': 1.8087,
 'theta_jn': 1.2473186485378833,
 'a_1': 0.1802,
 'a_2': 0.1681,
 'phase': 2.733968915528476,
 'geocent_time': 1886480670}

Waveform:

In [7]:
f_sampling = 4096.0
f_min = 20.0
duration = 24.0

waveform_arguments = {
    "waveform_approximant": "IMRPhenomPv2",
    "reference_frequency": f_min,
    "minimum_frequency": f_min
}

waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=f_sampling,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments
)

10:06 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_black_hole
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters


In [8]:
# Define the ET and CE detectors
ifos = bilby.gw.detector.InterferometerList(["ET", "CE"])

start_time = example_signal['tGPS'] - duration + 2

ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=f_sampling,
    duration=duration, 
    start_time=start_time)
ifos.inject_signal(waveform_generator=waveform_generator, parameters=example_signal)

/Users/Woute029/miniconda3/envs/overlapping_signals/lib/python3.10/site-packages/bilby/gw/source.py:528: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal
10:06 bilby INFO    : Injected signal in ET1:
10:06 bilby INFO    :   optimal SNR = 20.80
10:06 bilby INFO    :   matched filter SNR = 20.92+0.58j
10:06 bilby INFO    :   Mc = 20.456367915725753
10:06 bilby INFO    :   Phicoal = 0.4002946072018097
10:06 bilby INFO    :   chi1 = 0.1802
10:06 bilby INFO    :   chi1x

[{'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])},
 {'plus': array([0.-0.j, 0.-0.j, 0.-0.j, ..., 0.-0.j, 0.-0.j, 0.-0.j]),
  'cross': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j])}]